In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import matplotlib.pyplot as plt
from shapely import wkb

# Get the notebook directory and navigate to data folder
notebook_dir = os.path.dirname(os.path.abspath(__file__))
data_dir = os.path.join(os.path.dirname(notebook_dir), 'data')

In [ ]:
df_geo = gpd.read_file(os.path.join(data_dir, 'geo_pc4', 'georef-netherlands-postcode-pc4.geojson'))

DataSourceError: data\georef-netherlands-postcode-pc4.geojson: No such file or directory

In [ ]:
df_centroids = pd.read_csv(os.path.join(data_dir, 'new_job_data', 'd030_zones_met_centroids.csv'))  # this will later contain pc4 code for each zone_id

# Preparation

In [4]:
df_centroids['geometry'] = df_centroids['zone_geometrie'].apply(lambda x: wkb.loads(bytes.fromhex(x)))
gdf_centroids= gpd.GeoDataFrame(df_centroids, geometry='geometry')

In [5]:
gdf_centroids['geometry'] = gdf_centroids['geometry'].buffer(0)

In [6]:
gdf_centroids.set_crs(epsg=4326, inplace=True)

,gid,zone_id,zone_naam,gemeente_code,gemeente_naam,inwoners,inwoners_mannen,inwoners_vrouwen,inwoners_leeftijd_00_17_jaar,inwoners_leeftijd_18_29_jaar,...,arbeidsplaatsen_overige_gebruiksfunctie,arbeidsplaatsen_gecombineerde_functies,leerlingplaatsen_basisonderwijs,leerlingplaatsen_middelbaaronderwijs,leerlingplaatsen_middelbaarberoepsonderwijs,leerlingplaatsen_hogeronderwijs,stedelijkheidsgraad,centroid_geometrie,zone_geometrie,geometry
0,6746,BU05470103,Buitengebied,GM0547,Leiderdorp,343.518969,184.577356,158.941612,55.833998,46.833224,...,47.042003,30.971818,0.0,0.0,0.0,0.0,5.0,010100002040710000FD41CAACA5C8F740B265277DCA30...,010600002040710000010000000103000000010000004F...,"POLYGON ((98449.613 464612.326, 98450.775 4646..."
1,6747,BU05470200,Boterhuispolder,GM0547,Leiderdorp,34.007378,19.432787,14.574590,6.180860,6.122241,...,0.000000,4.830350,0.0,0.0,0.0,0.0,2.0,010100002040710000121710FFF65AF74050714A600B6C...,010600002040710000010000000103000000010000000C...,"POLYGON ((95618.428 466338.198, 95618.668 4663..."
2,6748,BU05470201,Buitenhof-Oost-Zuid,GM0547,Leiderdorp,1987.173039,933.671755,1053.501285,288.034882,203.714306,...,24.969446,105.550324,0.0,382.0,0.0,0.0,1.0,010100002040710000494C3569148CF74015C1F8AB5F5A...,010600002040710000010000000103000000010000004D...,"POLYGON ((96801.5238 464853.0375, 96808.7218 4..."
3,7076,BU06060107,Boylebuurt,GM0606,Schiedam,1225.369583,640.193088,585.176495,250.369010,250.344765,...,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,010100002040710000CDE8374D777DF5405F780EF991A8...,010600002040710000010000000103000000010000000C...,"POLYGON ((88149.9146 436832.1825, 88161.085 43..."
4,6820,BU05750000,De Zuid,GM0575,Noordwijk,710.823580,350.334479,360.489101,84.643888,80.081991,...,37.053715,39.392276,0.0,0.0,0.0,0.0,4.0,010100002040710000BED04A3EC4D7F5405DB40A036DD7...,01060000204071000001000000010300000001000000D0...,"POLYGON ((89677.9566 473043.2222, 89680.9222 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14417,10855,BU17050105,Buitengebied Gendt,GM1705,Lingewaard,1526.933798,793.212363,733.721435,256.363002,185.854402,...,7.395548,46.764052,0.0,0.0,0.0,0.0,5.0,01010000204071000061F8C9292DD10741012C06B35F6B...,010600002040710000010000000103000000030000005C...,"POLYGON ((193805.0108 434796.8823, 193810.6989..."
14418,1303,BU01470001,Borne Centrum,GM0147,Borne,3252.670921,1493.111690,1759.559231,308.644976,296.478310,...,8.464191,310.841607,0.0,0.0,0.0,0.0,2.0,0101000020407100001B760871794E0E41C6B29B4AD24C...,010600002040710000010000000103000000010000001F...,"POLYGON ((248581.9999 480486.5938, 248731.2031..."
14419,6742,BU05470010,De Bloemerd,GM0547,Leiderdorp,19.504231,9.752116,9.752116,3.772101,2.454573,...,0.000000,160.860224,0.0,0.0,0.0,0.0,3.0,01010000204071000092D3ED64CECCF740A8EE7F328256...,010600002040710000010000000103000000010000004D...,"POLYGON ((97206.384 465199.303, 97208.147 4651..."
14420,6744,BU05470101,Doeskwartier,GM0547,Leiderdorp,855.318889,407.534294,447.784595,146.691963,107.247637,...,0.000000,16.106516,0.0,0.0,0.0,0.0,3.0,0101000020407100002E26A8B5D88AF74096B3E5C03134...,010600002040710000010000000103000000010000003B...,"POLYGON ((96545.5475 462369.1345, 96559.4835 4..."


In [ ]:
# set the CRS for both geodatas to EPSG:4326
df_geo = df_geo.set_crs(gdf_centroids.crs, allow_override=True)

In [9]:
print(gdf_centroids.is_valid.all())  # Should return True
print(df_geo.is_valid.all()) 


True
True


In [10]:
print(gdf_centroids.total_bounds)  # [minx, miny, maxx, maxy]
print(df_geo.total_bounds)        

[ 13565.4000001  306846.19799995 278026.09000015 619231.64870024]
[ 3.35852377 50.75119188  7.22820712 53.51860333]


In [11]:
df_geo = df_geo.to_crs(epsg=28992)

In [12]:
df_geo['geometry'] = df_geo['geometry'].buffer(0)

In [13]:
len(df_geo)

4068

# Joining the pc5 with pc4
for all pc5 zones a centroid with 1 km buffer is made, for those pc5 which has their centeroid within a pc4 zone, a pc4 id is assigned.

In [ ]:

gdf_centroids['centroid'] = gdf_centroids.geometry.centroid


gdf_centroids_centroid = gpd.GeoDataFrame(
    gdf_centroids.drop(columns='geometry'), 
    geometry=gdf_centroids['centroid'].buffer(100),  # Use centroids with 100m buffer as geometry
    crs=gdf_centroids.crs                
)

gdf_joined = gpd.sjoin(
    gdf_centroids_centroid,
    df_geo[['pc4_code', 'geometry']],
    how='left',
    predicate='intersects'
)

# For zones with multiple PC4 codes, keep only the one with largest intersection area
if len(gdf_joined) > len(gdf_centroids_centroid):
    def calculate_intersection(row):
        matching_geometries = df_geo[df_geo['pc4_code'] == row['pc4_code']]['geometry']
        if len(matching_geometries) > 0:
            return row.geometry.intersection(matching_geometries.iloc[0]).area
        return 0

    gdf_joined['intersection_area'] = gdf_joined.apply(calculate_intersection, axis=1)
    
    gdf_joined = gdf_joined.sort_values(['zone_id', 'intersection_area'], ascending=[True, False])
    gdf_joined = gdf_joined.drop_duplicates('zone_id')
    
    gdf_joined = gdf_joined.drop(columns=['intersection_area'])


C:\Users\yeslamijadidi\AppData\Local\Temp\ipykernel_11092\171502921.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_centroids['centroid'] = gdf_centroids.geometry.centroid
C:\Users\yeslamijadidi\AppData\Local\Temp\ipykernel_11092\171502921.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometry=gdf_centroids['centroid'].buffer(100),  # Use centroids with 100m buffer as geometry
C:\Users\yeslamijadidi\AppData\Local\Temp\ipykernel_11092\171502921.py:11: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:28992

  gdf_joined = gpd.sjoin(


only 14 of them are not assigned to any pc4 which is acceptable

In [15]:
gdf_joined[gdf_joined['pc4_code'].isna()]

,gid,zone_id,zone_naam,gemeente_code,gemeente_naam,inwoners,inwoners_mannen,inwoners_vrouwen,inwoners_leeftijd_00_17_jaar,inwoners_leeftijd_18_29_jaar,...,leerlingplaatsen_middelbaaronderwijs,leerlingplaatsen_middelbaarberoepsonderwijs,leerlingplaatsen_hogeronderwijs,stedelijkheidsgraad,centroid_geometrie,zone_geometrie,centroid,geometry,index_right,pc4_code
1299,482,BU00720007,Industriehaven en omgeving,GM0072,Harlingen,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,5.0,0101000020407100001D68F9DA9B41034116180D44D8A3...,010600002040710000020000000103000000010000006A...,POINT (156820.05793 577675.6107),"POLYGON ((156920.05793 577675.6107, 156919.576...",NaN,NaN
13966,2720,BU02430204,De Knar,GM0243,Harderwijk,2.000450,0.000000,2.000450,0.544899,0.201538,...,0.0,0.0,0.0,4.0,010100002040710000B9A14D25D9D30441D473C24C54A5...,010600002040710000070000000103000000010000008F...,POINT (169583.96355 485968.19894),"POLYGON ((169683.96355 485968.19894, 169683.48...",NaN,NaN
14174,4551,BU0363MK01,Buiteneiland,GM0363,Amsterdam,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,5.0,010100002040710000A143503CFF85FF4049F79993E3B3...,010600002040710000010000000103000000010000001B...,POINT (129119.95222 486648.89414),"POLYGON ((129219.95222 486648.89414, 129219.47...",NaN,NaN
959,4803,BU03760107,Eemmeer-Blaricum,GM0376,Blaricum,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,5.0,01010000204071000088D8F4C0E45202410E4FC8657535...,010600002040710000030000000103000000010000004B...,POINT (149836.73516 478654.04909),"POLYGON ((149936.73516 478654.04909, 149936.25...",NaN,NaN
14208,4931,BU03881002,Vogeleiland,GM0388,Enkhuizen,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,5.0,01010000204071000051DD6863459B014140DBBF8A9450...,0106000020407100000700000001030000000100000016...,POINT (144232.67354 534602.27099),"POLYGON ((144332.67354 534602.27099, 144332.19...",NaN,NaN
5622,5383,BU04053400,Hoorn 80 - Buurt 34 00,GM0405,Hoorn,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,4.0,010100002040710000A8A61F22266A0041A11146717C92...,0106000020407100000100000001030000000100000044...,POINT (134705.12384 517237.84607),"POLYGON ((134805.12384 517237.84607, 134804.64...",NaN,NaN
158,7756,BU07170209,Verspreide huizen Vrouwenpolder,GM0717,Veere,277.343209,138.671605,138.671605,53.579976,45.481761,...,0.0,0.0,0.0,5.0,010100002040710000069528A35EF7DF40AB9829B78A7B...,010600002040710000020000000103000000020000004C...,POINT (33282.34513 401928.04148),"POLYGON ((33382.34513 401928.04148, 33381.8636...",NaN,NaN
252,9885,BU09950651,Houtribhoogte Zuid,GM0995,Lelystad,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,4.0,0101000020407100006CF89CBEA458034125A18AABB8C2...,010600002040710000050000000103000000010000006C...,POINT (154636.06447 512751.64003),"POLYGON ((154736.06447 512751.64003, 154735.58...",NaN,NaN
10130,10514,BU16760501,Recreatiegebied,GM1676,Schouwen-Duiveland,372.923152,193.159079,179.764074,22.793778,28.655210,...,0.0,0.0,0.0,5.0,010100002040710000AB1AB3FE5B6AEF40D4C241E3AB05...,0106000020407100000B00000001030000000100000022...,POINT (67665.38112 409364.18024),"POLYGON ((67765.38112 409364.18024, 67764.8995...",NaN,NaN
13486,12287,BU19001503,Kornwerderzand,GM1900,Súdwest-Fryslân,3.002842,1.513985,1.488857,0.573441,0.383285,...,0.0,0.0,0.0,5.0,010100002040710000918F227EF09E0141F971E558271B...,01060000204071000001000000010300000001000000EB...,POINT (145735.83613 561534.32604),"POLYGON ((145835.83613 561534.32604, 145835.35...",NaN,NaN


In [ ]:

df_geo_codes = set(df_geo['pc4_code'])
gdf_joined_codes = set(gdf_joined['pc4_code'])

missing_codes = df_geo_codes - gdf_joined_codes

print(f"Number of missing codes: {len(missing_codes)}")
print("\nMissing pc4_codes:")
print(sorted(missing_codes))

Number of missing codes: 151

Missing pc4_codes:
['1117', '1189', '1428', '1436', '1485', '1642', '1663', '1682', '1683', '1686', '1692', '1847', '1873', '2019', '2212', '2216', '2362', '2523', '2546', '2561', '2563', '2693', '2726', '2727', '2872', '2989', '3022', '3036', '3052', '3064', '3069', '3078', '3087', '3191', '3193', '3292', '3293', '3546', '3653', '3754', '3895', '4211', '4212', '4302', '4303', '4482', '4522', '4652', '4722', '4781', '4835', '5028', '5252', '5272', '5292', '5293', '5302', '5318', '5383', '5449', '5471', '5689', '5722', '6092', '6111', '6237', '6251', '6278', '6311', '6341', '6421', '6469', '6532', '6572', '6584', '7046', '7105', '7106', '7216', '7392', '7472', '7525', '7536', '7675', '7813', '7821', '7859', '7877', '7892', '7911', '7915', '7928', '7937', '7984', '7985', '8052', '8153', '8154', '8254', '8291', '8363', '8438', '8444', '8469', '8489', '8631', '8702', '8752', '8755', '8766', '8871', '8882', '8883', '8885', '8892', '8939', '9085', '9102', '9111'

In [17]:
gdf_joined['geometry'] = gdf_centroids['geometry']

In [18]:
gdf_centroids = gdf_joined

In [19]:
gdf_centroids.drop(columns=['centroid'], inplace=True)

In [20]:
gdf_centroids[['gid', 'pc4_code', 'zone_id']]

,gid,pc4_code,zone_id
1055,1,9712,BU00140000
1056,2,9711,BU00140001
1047,3,9711,BU00140002
1048,4,9718,BU00140003
1051,5,9712,BU00140004
...,...,...,...
13041,14418,3238,BU19923200
13051,14419,3238,BU19923201
13054,14420,3238,BU19923202
13120,14421,3238,BU19923209


In [ ]:
pc4_code_mapping = {
    1117: 1432,
    9999: 9959,
    3292: 3291,
    3078: 3079,
    8052: 8051
}

for old_code, new_code in pc4_code_mapping.items():
    gdf_centroids.loc[gdf_centroids['pc4_code'] == old_code, 'pc4_code'] = new_code

In [ ]:
gdf_centroids[['gid', 'pc4_code', 'zone_id']].to_csv(os.path.join(data_dir, 'geo_coupled', 'centroids_with_pc4_v5.csv'), index=False)

writing the file 

In [ ]:
gdf_centroids.to_file(os.path.join(data_dir, 'geo_coupled', 'centroids_with_pc4_v5.shp'))

C:\Users\yeslamijadidi\AppData\Local\Temp\ipykernel_11092\4076425810.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_centroids.to_file(r"C:\Project\Test\forYasi\centroids_with_pc4\centroids_with_pc4_v5.shp")
C:\Users\yeslamijadidi\AppData\Roaming\Python\Python312\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'gemeente_code' to 'gemeente_c'
  ogr_write(
C:\Users\yeslamijadidi\AppData\Roaming\Python\Python312\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'gemeente_naam' to 'gemeente_n'
  ogr_write(
C:\Users\yeslamijadidi\AppData\Roaming\Python\Python312\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'inwoners_mannen' to 'inwoners_m'
  ogr_write(
C:\Users\yeslamijadidi\AppData\Roaming\Python\Python312\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'inwoners_vrouwen' to 'inwoners_v'
